In [ ]:
# Activar Google Drive.
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
# Entrena un modelo de Gradient Boosted Trees
# para predecir el consumo eléctrico
# y la cantidad de usuarios.

# Entrena el modelo final utilizado para el paper.

import pandas as pd
import numpy as np

import xgboost as xgb

from joblib import dump

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# Información

In [ ]:
path_drive = "/content/drive/MyDrive/Colab/Subsidio_electricidad/" 
path_data = path_drive + "data/Actual/data"
fname = ".csv"

data_0 = pd.read_csv(path_data + fname)

# Unimos las tarifas 1 y DAC.
data_0["Consumo_1*"] += data_0["Consumo_DAC"]
data_0["Usuarios_1*"] += data_0["Usuarios_DAC"]
data_0.drop( ["Consumo_DAC", "Usuarios_DAC"], axis = 1, inplace = True )

# Escogemos las variables relevantes.
data_0.drop( ["CVE_MUN", "Tmax", "Tmin", "HDD_mean", "HDD_p10", 
    "CDD_p90", "$luz", "Tmean_max_2", "M_verano", "Verano"],
    axis = 1, inplace = True)

# Promediamos todos los municipios
var_area = ["Tmean", "Pre", "CDD_mean", "Pre_Tmean", "$GLP"]
data_0[var_area] = ( data_0[var_area]
    * np.expand_dims( data_0["Area"].values, 1 ) )
data_0 = data_0.groupby(["CVE_ENT", "Año", "Mes"]).sum()
data_0[var_area] = ( data_0[var_area]
    / np.expand_dims( data_0["Area"].values, 1 ) )
data_0["PCI"] = data_0["PIB"] / data_0["Poblacion"]
data_0["Densidad_poblacion"] = data_0["Poblacion"] / data_0["Area"]
data_0[["NOM_ENT", "NOM_MUN", "Tarifa"]] = None
data_0.reset_index(inplace = True)

# Reordenamos las columnas
data_0 = data_0[ list(data_0.columns[3:4]) + list(data_0.columns[0:1]) 
    + list(data_0.columns[-3:-1]) + list(data_0.columns[4:6])
    + list(data_0.columns[-1:]) + list(data_0.columns[6:-3])
    + list(data_0.columns[1:3]) ]

# Escogemos las variables relevantes.
data_0.drop( "Area", axis = 1, inplace = True)

# Creamos columnas de apoyo.
data_0["Consumo_Usuario"] = data_0["Consumo_1*"] / data_0["Usuarios_1*"]
data_0["Usuario_poblacion"] = data_0["Usuarios_1*"] / data_0["Poblacion"]

# Límites para corrección.
lim = [0.1, 0.6, 30] 

# Estadísticas de corrección.
# Porcentaje de municipios con corrección.
a = data_0.loc[ data_0["Usuario_poblacion"] < lim[0] ].shape[0]
b = data_0.loc[ data_0["Usuario_poblacion"] > lim[1] ].shape[0]
c = data_0.loc[ data_0["Consumo_Usuario"]   < lim[2] ].shape[0]
d = ( a + b + c ) / 12 / 7 / 2469 * 100
print(f"Porcentaje de municipios con corrección: {d:.2f}")
# Porcentaje de población con corrección.
a = data_0.loc[ data_0["Usuario_poblacion"] < lim[0], "Poblacion" ].sum()
b = data_0.loc[ data_0["Usuario_poblacion"] > lim[1], "Poblacion" ].sum()
c = data_0.loc[ data_0["Consumo_Usuario"]   < lim[2], "Poblacion" ].sum()
d = ( a + b + c ) / 12 / 7 / data_0["Poblacion"].sum() * 100
print(f"Porcentaje de población con corrección:  {d:.2f}")
# Porcentaje de consumo con corrección.
a = data_0.loc[ data_0["Usuario_poblacion"] < lim[0], "Consumo_1*" ].sum()
b = data_0.loc[ data_0["Usuario_poblacion"] > lim[1], "Consumo_1*" ].sum()
c = data_0.loc[ data_0["Consumo_Usuario"]   < lim[2], "Consumo_1*" ].sum()
d = ( a + b + c ) / 12 / 7 / data_0["Poblacion"].sum() * 100
print(f"Porcentaje de consumo con corrección:    {d:.2f}")

# Correcciones de municipios que están abajo o arriba de los límites.
data_0.loc[data_0["Usuario_poblacion"] < lim[0], "Usuarios_1*"] = (
    data_0.loc[data_0["Usuario_poblacion"] < lim[0], "Poblacion"] * lim[0] )
data_0.loc[data_0["Usuario_poblacion"] > lim[1], "Usuarios_1*"] = (
    data_0.loc[data_0["Usuario_poblacion"] > lim[1], "Poblacion"] * lim[1] )
data_0.loc[data_0["Consumo_Usuario"] < lim[2], "Consumo_1*"] = (
data_0.loc[data_0["Consumo_Usuario"] < lim[2], "Usuarios_1*"] * 10 )
data_0["Consumo_Usuario"] = data_0["Consumo_1*"] / data_0["Usuarios_1*"]
data_0["Usuario_poblacion"] = data_0["Usuarios_1*"] / data_0["Poblacion"]

# Consumo y usuarios totales en 2010.
cons = data_0[data_0["Año"] == 2010]['Consumo_1*'].sum()
us = data_0[data_0["Año"] == 2010].groupby('CVE_INEGI'
    ).mean()['Usuarios_1*'].sum()

# Información utilizada para el modelo.
data_0.head()

Porcentaje de municipios con corrección: 0.00
Porcentaje de población con corrección:  0.00
Porcentaje de consumo con corrección:    0.00


,CVE_INEGI,CVE_ENT,NOM_ENT,NOM_MUN,Consumo_1*,Usuarios_1*,Tarifa,lon,lat,Tmean,...,Pre_Tmean,Poblacion,PIB,Densidad_poblacion,PCI,$GLP,Año,Mes,Consumo_Usuario,Usuario_poblacion
0,11066,1,None,None,3.184341e+07,337089.0,None,-1125.413944,242.84132,13.880437,...,0.000000,1184996.0,1.523398e+11,2.131796,128557.232252,10.151,2010,1,94.465873,0.284464
1,11066,1,None,None,2.922356e+07,337089.0,None,-1125.413944,242.84132,14.177117,...,0.000000,1184996.0,1.523398e+11,2.131796,128557.232252,10.151,2010,2,86.693907,0.284464
2,11066,1,None,None,3.296324e+07,337089.0,None,-1125.413944,242.84132,17.059910,...,0.837011,1184996.0,1.523398e+11,2.131796,128557.232252,10.151,2010,3,97.787935,0.284464
3,11066,1,None,None,3.592753e+07,337089.0,None,-1125.413944,242.84132,19.016532,...,1.688246,1184996.0,1.523398e+11,2.131796,128557.232252,10.151,2010,4,106.581727,0.284464
4,11066,1,None,None,3.709735e+07,337089.0,None,-1125.413944,242.84132,23.460577,...,10.580766,1184996.0,1.523398e+11,2.131796,128557.232252,10.151,2010,5,110.052100,0.284464


In [ ]:
# Estadísticos de las variables.
data_0.describe()

,CVE_INEGI,CVE_ENT,Consumo_1*,Usuarios_1*,lon,lat,Tmean,Pre,CDD_mean,Pre_Tmean,Poblacion,PIB,Densidad_poblacion,PCI,$GLP,Año,Mes,Consumo_Usuario,Usuario_poblacion
count,2.688000e+03,2688.000000,2.688000e+03,2.688000e+03,2688.000000,2688.000000,2688.000000,2688.000000,2688.000000,2688.000000,2.688000e+03,2.688000e+03,2688.000000,2688.000000,2688.000000,2688.000000,2688.000000,2688.000000,2688.000000
mean,1.490980e+06,16.500000,1.386414e+08,1.038275e+06,-7618.642581,1544.885215,22.789262,75.634836,45.819134,52.275394,3.646668e+06,5.219837e+11,2.950229,156903.039837,11.048164,2013.000000,6.500000,145.061854,0.295400
std,2.265640e+06,9.234811,9.729661e+07,7.705185e+05,10033.483440,1802.829671,4.578247,90.296344,53.720546,68.743924,3.032505e+06,5.229642e+11,10.310719,130756.286911,0.909136,2.000372,3.452695,73.758051,0.028866
min,1.106600e+04,1.000000,2.289427e+07,1.985030e+05,-55205.929247,126.195374,8.334948,0.000000,0.000000,0.000000,6.370260e+05,8.199218e+10,0.086119,53666.233993,8.948300,2010.000000,1.000000,57.030039,0.210366
25%,3.334240e+05,8.750000,5.547413e+07,4.869150e+05,-8588.066821,358.094293,19.401556,10.826798,3.644967,1.626418,1.623929e+06,2.000017e+11,0.378756,94824.429244,10.269594,2011.000000,3.750000,94.507620,0.274233
50%,7.442230e+05,16.500000,1.130852e+08,8.481330e+05,-4871.942374,1100.760929,23.161116,35.109865,26.333645,15.429780,2.825344e+06,3.372610e+11,0.605288,124539.341770,11.153076,2013.000000,6.500000,110.730015,0.296069
75%,1.772016e+06,24.250000,2.106820e+08,1.343340e+06,-1745.374889,2066.917114,26.346781,117.116900,68.834125,91.887707,4.636732e+06,5.659993e+11,1.446275,170597.539417,11.780569,2015.000000,9.250000,183.286450,0.315320
max,1.156274e+07,32.000000,4.784287e+08,4.211257e+06,-556.908984,9741.879913,32.324915,591.825043,275.980610,447.724318,1.638996e+07,2.958634e+12,60.101385,901819.117320,13.169542,2016.000000,12.000000,512.095931,0.378262


In [ ]:
# Matriz de correlación de Pearson.
data_0.corr()

,CVE_INEGI,CVE_ENT,Consumo_1*,Usuarios_1*,lon,lat,Tmean,Pre,CDD_mean,Pre_Tmean,Poblacion,PIB,Densidad_poblacion,PCI,$GLP,Año,Mes,Consumo_Usuario,Usuario_poblacion
CVE_INEGI,1.000000e+00,3.941221e-01,0.096817,2.634859e-01,-9.583334e-01,9.611071e-01,0.097026,0.153662,-0.078932,0.116449,2.430222e-01,-2.867944e-02,-1.047475e-01,-2.504945e-01,-3.558609e-02,-7.155399e-14,-1.481996e-16,-0.233677,-1.317042e-01
CVE_ENT,3.941221e-01,1.000000e+00,0.128181,2.139835e-02,-2.170847e-01,2.508829e-01,0.144120,0.113762,-0.025603,0.102900,1.546555e-03,-1.153100e-01,-1.425839e-01,-2.660426e-01,-1.188315e-02,-6.324700e-14,-2.091613e-16,0.048570,5.338740e-02
Consumo_1*,9.681733e-02,1.281808e-01,1.000000,7.077125e-01,-8.590420e-02,1.282783e-01,0.096473,0.020228,0.289194,0.064980,6.529368e-01,5.915344e-01,2.490591e-01,-9.484090e-03,-5.468500e-02,7.499791e-02,6.271886e-02,0.475088,-3.563299e-02
Usuarios_1*,2.634859e-01,2.139835e-02,0.707712,1.000000e+00,-3.176155e-01,3.354335e-01,-0.180908,0.051262,-0.039269,0.063671,9.859994e-01,7.424683e-01,4.180433e-01,-1.106836e-01,-1.802528e-02,7.400862e-02,-1.256589e-16,-0.210747,-3.542220e-01
lon,-9.583334e-01,-2.170847e-01,-0.085904,-3.176155e-01,1.000000e+00,-9.942032e-01,-0.053922,-0.147827,0.084518,-0.106908,-3.059078e-01,-6.306751e-03,1.022157e-01,2.580530e-01,4.554356e-02,4.932317e-14,4.158380e-17,0.299416,2.122976e-01
lat,9.611071e-01,2.508829e-01,0.128278,3.354335e-01,-9.942032e-01,1.000000e+00,0.043074,0.132878,-0.060466,0.098551,3.199878e-01,1.439144e-02,-1.191045e-01,-2.659014e-01,-4.689528e-02,-4.691617e-14,-6.330906e-17,-0.272512,-2.074172e-01
Tmean,9.702648e-02,1.441199e-01,0.096473,-1.809076e-01,-5.392194e-02,4.307440e-02,1.000000,0.378215,0.568396,0.417820,-1.939873e-01,-1.813915e-01,-2.318473e-01,1.099717e-01,2.969161e-02,3.903072e-02,9.795852e-02,0.418148,2.550109e-01
Pre,1.536620e-01,1.137621e-01,0.020228,5.126240e-02,-1.478268e-01,1.328780e-01,0.378215,1.000000,0.092093,0.908159,5.892077e-02,2.486756e-03,-2.487392e-03,3.570525e-02,-2.305995e-02,-2.237061e-03,2.388809e-01,-0.014153,-1.051416e-01
CDD_mean,-7.893195e-02,-2.560333e-02,0.289194,-3.926931e-02,8.451808e-02,-6.046559e-02,0.568396,0.092093,1.000000,0.267843,-5.550283e-02,-2.057800e-02,-7.502525e-02,4.048989e-04,-1.430815e-02,1.824780e-02,3.607976e-02,0.418705,8.236255e-02
Pre_Tmean,1.164488e-01,1.029003e-01,0.064980,6.367064e-02,-1.069077e-01,9.855104e-02,0.417820,0.908159,0.267843,1.000000,6.664327e-02,8.928363e-03,3.255540e-03,3.052607e-03,6.880645e-04,2.604985e-02,2.028494e-01,0.018268,-8.131547e-02


# Usuarios

In [ ]:
# Predicción de usuarios.

# Escogemos el conjunto de features y de variables a predecir.
Y = data_0.iloc[:,5]
X = data_0[ ["Poblacion", "PIB", "PCI", "Año"] ].copy()
X = pd.concat( (X, pd.get_dummies(data_0["CVE_ENT"]) ), axis = 1)

# Separamos en conjuntos de entrenamiento y de prueba.
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size = 0.3 )

# Separamos en conjunto de prueba y de validación.
X_train, X_val, Y_train, Y_val = train_test_split( X, Y, test_size = 0.5 )

# Features a utilizar.
X.head()

,Poblacion,PIB,PCI,Año,1,2,3,4,5,6,...,23,24,25,26,27,28,29,30,31,32
0,1184996.0,1.523398e+11,128557.232252,2010,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1184996.0,1.523398e+11,128557.232252,2010,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1184996.0,1.523398e+11,128557.232252,2010,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1184996.0,1.523398e+11,128557.232252,2010,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1184996.0,1.523398e+11,128557.232252,2010,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Variable a predecir.
Y.head()

0    337089.0
1    337089.0
2    337089.0
3    337089.0
4    337089.0
Name: Usuarios_1*, dtype: float64

In [ ]:
# Entrenamiento simple.

# Copiamos las variables para poder
# transformarlas en caso de ser necesario.
X_reg = X_train.copy()
Y_reg = Y_train.copy()
X_reg_val = X_val.copy()
X_reg_test = X_test.copy()

# Hiperparámetros.
params = {
    "objective": "reg:squarederror",
    "colsample_bytree": 0.3,
    "learning_rate": 0.1,
    "max_depth": 50,
    "min_child_weight" : 25,
    "min_split_loss": 0.2,
    "n_estimators": 100
    }

# Creamos el regresor con los hiperparámetros.
xg_reg_u = xgb.XGBRegressor( **params )

# Entrenamos el modelo.
xg_reg_u.fit(X_reg, Y_reg, verbose = True)

# Probamos la regresión en el set de entrenamiento.
preds = xg_reg_u.predict(X_reg)

# Calculamos el error de entrenamiento.
rmse = np.sqrt(mean_squared_error(Y_reg, preds))
r2 = r2_score(Y_reg, preds)

print("Entrenamiento")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

Entrenamiento
RMSE: 1.469E+04
R^2:  0.9996508


In [ ]:
# Probamos la regresión en el set de validación.
preds = xg_reg_u.predict(X_reg_val)

# Calculamos el error de validación.
rmse = np.sqrt(mean_squared_error(Y_val, preds))
r2 = r2_score(Y_val, preds)

print("Validación")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Probamos la regresión en el set de prueba
# y guardamos el resultado para su evaluación final.
test_0 = xg_reg_u.predict(X_reg_test)

Validación
RMSE: 1.470E+04
R^2:  0.9996200


In [ ]:
# Calculamos el error de prueba.
rmse = np.sqrt(mean_squared_error(Y_test, test_0))
r2 = r2_score(Y_test, test_0)

print("Prueba")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Guardamos el modelo entrenado.
#dump(xg_reg_u, path_drive + "reg_model_usuarios.joblib")

Prueba
RMSE: 1.539E+04
R^2:  0.9996017


# Consumo por usuario

In [ ]:
# Predicción de consumo por usuario.
# Escogemos el conjunto de features y de variables a predecir.
X, Y = data_0.iloc[:,7:-2], data_0.iloc[:,-2]

# Quitamos algunas features.
X.drop( ["Pre", "Año", "Poblacion", "PIB", "lat", "lon"],
    axis = 1, inplace = True )
X = pd.concat( (X, pd.get_dummies(data_0["CVE_ENT"]) ), axis = 1)

# Separamos en conjuntos de entrenamiento y de prueba.
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size = 0.3 )

# Separamos en conjunto de prueba y de validación.
X_train, X_val, Y_train, Y_val = train_test_split( X, Y, test_size = 0.5 )

# Features a utilizar.
X.head()

,Tmean,CDD_mean,Pre_Tmean,Densidad_poblacion,PCI,$GLP,Mes,1,2,3,...,23,24,25,26,27,28,29,30,31,32
0,13.880437,0.000000,0.000000,2.131796,128557.232252,10.151,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,14.177117,0.000000,0.000000,2.131796,128557.232252,10.151,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,17.059910,0.918227,0.837011,2.131796,128557.232252,10.151,3,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,19.016532,5.279840,1.688246,2.131796,128557.232252,10.151,4,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,23.460577,121.287589,10.580766,2.131796,128557.232252,10.151,5,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Variable a predecir.
Y.head()

0     94.465873
1     86.693907
2     97.787935
3    106.581727
4    110.052100
Name: Consumo_Usuario, dtype: float64

In [ ]:
# Entrenamiento simple.

# Copiamos las variables para poder
# transformarlas en caso de ser necesario.
X_reg = X_train.copy()
Y_reg = Y_train.copy()
X_reg_val = X_val.copy()
X_reg_test = X_test.copy()

# Hiperparámetros.
params = {
    "objective": "reg:squarederror",
    "colsample_bytree": 0.3,
    "learning_rate": 0.1,
    "max_depth": 50,
    "min_child_weight" : 25,
    "min_split_loss": 0.2,
    "n_estimators": 100
    }

# Creamos el regresor con los hiperparámetros.
xg_reg = xgb.XGBRegressor( **params )

# Entrenamos el modelo.
xg_reg.fit(X_reg, Y_reg, verbose = True)

# Probamos la regresión en el set de entrenamiento.
preds = xg_reg.predict(X_reg)

# Calculamos el error de entrenamiento.
rmse = np.sqrt(mean_squared_error(Y_reg, preds))
r2 = r2_score(Y_reg, preds)

print("Entrenamiento")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

Entrenamiento
RMSE: 8.873E+00
R^2:  0.9857962


In [ ]:
# Probamos la regresión en el set de validación.
preds = xg_reg.predict(X_reg_val)

# Calculamos el error de validación.
rmse = np.sqrt(mean_squared_error(Y_val, preds))
r2 = r2_score(Y_val, preds)

print("Validación")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Probamos la regresión en el set de prueba
# y guardamos el resultado para su evaluación final.
test = xg_reg.predict(X_reg_test)

Validación
RMSE: 1.489E+01
R^2:  0.9584394


In [ ]:
# Calculamos el error de prueba.
rmse = np.sqrt(mean_squared_error(Y_test, test))
r2 = r2_score(Y_test, test)

print("Prueba")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Guardamos el modelo entrenado.
#dump(xg_reg, path_drive + "reg_model_consumo.joblib")

Prueba
RMSE: 1.197E+01
R^2:  0.9716191


# Predicción

In [ ]:
# Datos.
path = "/content/drive/MyDrive/Colab/Subsidio_electricidad/" 
path_data = path + "data/"
fname = "data.csv"

# Escenarios
RCP = ["RCP4p5", "RCP8p5"]
fut = [2030, 2050, 2070]

# Directorios.
#dirs = ["Actual"] + RCP
dirs = [RCP[1]]
csv = ".csv"
path_data = [path_data + x + "/data." for x in dirs]
f_path = []
for p in path_data:
    f_path = ( f_path + [p + str(x) + csv for x in fut]
        + [p + str(x) + "_tarifa_teorica" + csv for x in fut]
        + [p + str(x) + "_pob_pib" + csv for x in fut] )
    
# Nombres de los archivos con la información de los estados.
f_path_estados = [ x[:-4] + "_estados.csv" for x in f_path ]

In [ ]:
reg_model_pred = [ xg_reg, xg_reg_u ]

# Iteramos para todos los escenarios.
for n, i in enumerate(f_path):
    data = pd.read_csv(i)

    if   i == f_path[6]: data["Año"] = 2030
    elif i == f_path[7]: data["Año"] = 2050
    elif i == f_path[8]: data["Año"] = 2070

    # Promediamos todos los municipios
    var_area = ["Tmean", "Pre", "CDD_mean", "Pre_Tmean", "$GLP"]
    data[var_area] = ( data[var_area]
        * np.expand_dims( data["Area"].values, 1 ) )
    data = data.groupby(["CVE_ENT", "Año", "Mes"]).sum()
    data[var_area] = ( data[var_area]
        / np.expand_dims( data["Area"].values, 1 ) )
    data["PCI"] = data["PIB"] / data["Poblacion"]
    data["Densidad_poblacion"] = data["Poblacion"] / data["Area"]
    data[["NOM_ENT", "NOM_MUN", "Tarifa"]] = None
    data.reset_index(inplace = True)

    # Escogemos el conjunto de features para
    # la predicción de consumo por usuario.
    X = data[ ["Tmean", "CDD_mean", "Pre_Tmean",
        "Densidad_poblacion", "PCI", "$GLP", "Mes"] ].copy()
    X = pd.concat( (X, pd.get_dummies(data["CVE_ENT"]) ), axis = 1)

    # Escogemos el conjunto de features para la predicción de usuarios.
    X_u = data[ ["Poblacion", "PIB", "PCI", "Año"] ].copy()
    X_u = pd.concat( (X_u, pd.get_dummies(data["CVE_ENT"]) ), axis = 1)
    
    # Hacemos la predicción y calculamos el consumo total.
    data["Consumo_Usuario"] = reg_model_pred[0].predict(X)
    data["Usuarios_1*"] = reg_model_pred[1].predict(X_u)
    data["Consumo_1*"] = data["Usuarios_1*"] * data["Consumo_Usuario"]
    data["Usuario_poblacion"] = data["Usuarios_1*"] / data["Poblacion"]
    
    # Nombre de la corrida.
    print(f"Corrida: {i[68:-4]}")
    # Límites para corrección.
    lim = [0.11, 1.6, 30] 

    # Estadísticas de corrección.
    # Porcentaje de municipios abajo o arriba del umbral.
    a = ( ( data[ data["Usuario_poblacion"] < lim[0] ].shape[0]
        +   data[ data["Usuario_poblacion"] > lim[1] ]["Poblacion"].shape[0] )
        / 12 / 2649 * 100 )
    #print(f"Porcentaje de municipios abajo o arriba del umbral:     {a:05.2f}")
    # Porcentaje de la población abajo o arriba del umbral.
    a = ( ( data.loc[ data["Usuario_poblacion"] < lim[0], "Poblacion"].sum()
        +   data.loc[ data["Usuario_poblacion"] > lim[1], "Poblacion"].sum() )
        / data["Poblacion"].sum() * 100 )
    #print(f"Porcentaje de la población abajo o arriba del umbral:    {a:.2f}")
    # Porcentaje del consumo abajo o arriba del umbral.
    a = ( ( data_0.where( data["Usuario_poblacion"] > lim[1] ).dropna()
        .groupby('CVE_INEGI').mean()["Consumo_1*"].sum() + 
        ( data_0.where( data["Usuario_poblacion"] < lim[0] ).dropna()
        .groupby('CVE_INEGI').mean()["Consumo_1*"].sum() ) ) * 100 / cons )
    #print(f"Porcentaje del consumo abajo o arriba del umbral:        {a:.2f}")
    # Porcentaje del consumo arriba del umbral.
    a = data[ data["Usuario_poblacion"] > lim[1] ]
    a = ( ( ( a["Usuario_poblacion"] - lim[1] ) * a["Consumo_Usuario"]
        * a["Poblacion"] ).sum() / data["Consumo_1*"].sum() * 100 )
    #print(f"Porcentaje del consumo arriba del umbral:                {a:.2f}")
    # Porcentaje de municipios que tuvieron consumo negativo.
    #a = data[ data["Usuarios_1*"] < 0 ].shape[0] * 100 / 12 / 2469
    #print(f"Porcentaje de municipios que tuvieron consumo negativo:  {a:.2f}")
    # Porcentaje de población que tuvo consumo negativo.
    #a = ( data.loc[ data["Usuarios_1*"] < 0, "Poblacion"].sum()
    #    / data["Poblacion"].sum() * 100 )
    #print(f"Porcentaje de población que tuvo consumo negativo:       {a:.2f}")
    # Porcentaje de municipios abajo del umbral de Consumo_Usuario.
    a = data[ data["Consumo_Usuario"] < lim[2] ].shape[0] / 12 / 2469 * 100 
    #print(f"Porcentaje de municipios abajo del umbral de consumo:    {a:.2f}")

    # Correcciones de municipios que están abajo o arriba de los límites.
    data.loc[ data["Usuario_poblacion"] < lim[0], "Usuarios_1*" ] = (
        data.loc[ data["Usuario_poblacion"] < lim[0], "Poblacion" ] * lim[0] )
    data.loc[ data["Usuario_poblacion"] > lim[1], "Usuarios_1*" ] = (
        data.loc[ data["Usuario_poblacion"] > lim[1], "Poblacion" ] * lim[1] )
    data["Consumo_1*"] = data["Usuarios_1*"] * data["Consumo_Usuario"]
    data.loc[ data["Consumo_Usuario"]   < lim[2], "Consumo_1*" ] = (
        data.loc[ data["Consumo_Usuario"]   < lim[2], "Usuarios_1*" ] * 10 )

    # Retiramos las columnas de apoyo.
    data.drop( columns = ["Consumo_Usuario", "Usuario_poblacion"],
        inplace = True )

    # Incremento en consumo con respecto a 2010.
    perc = ( data['Consumo_1*'].sum() / cons - 1 ) * 100
    print(f"Consumo:  {data['Consumo_1*'].sum():+.2e}, {perc:06.2f}%")
    # Incremento en usuarios con respecto a 2010.
    a = data.groupby('CVE_INEGI').mean()['Usuarios_1*'].sum()
    perc = ( a / us - 1 ) * 100
    print(f"Usuarios: {a:+.2e}, {perc:06.2f}% \n")
    
    # Guardamos el archivo.
    data.to_csv(f_path_estados[n], index = False)

Corrida: 2030
Consumo:  +7.09e+10, 045.55%
Usuarios: +3.85e+07, 025.54% 

Corrida: 2050
Consumo:  +7.24e+10, 048.67%
Usuarios: +4.10e+07, 034.01% 

Corrida: 2070
Consumo:  +8.16e+10, 067.58%
Usuarios: +4.08e+07, 033.25% 

Corrida: 2030_tarifa_teorica
Consumo:  +7.09e+10, 045.55%
Usuarios: +3.85e+07, 025.54% 

Corrida: 2050_tarifa_teorica
Consumo:  +7.24e+10, 048.67%
Usuarios: +4.10e+07, 034.01% 

Corrida: 2070_tarifa_teorica
Consumo:  +8.16e+10, 067.58%
Usuarios: +4.08e+07, 033.25% 

Corrida: 2030_pob_pib
Consumo:  +6.79e+10, 039.44%
Usuarios: +3.85e+07, 025.54% 

Corrida: 2050_pob_pib
Consumo:  +6.58e+10, 035.00%
Usuarios: +4.10e+07, 034.01% 

Corrida: 2070_pob_pib
Consumo:  +7.01e+10, 043.90%
Usuarios: +4.08e+07, 033.25% 

